In [1]:
from scadaUtils import comUtils
import os,sys,re, pandas as pd

# 1.Build the configuration of your project using the Conf_generator
Let's say we have the following configuration. Some 2 devicecs in modbus and one in opcua.

In [2]:
dummy_df_devices = pd.read_excel('data/dummy_devices.ods', sheet_name = 'devices', index_col=0)
dummy_df_devices

,protocole,IP,byte_order,word_order,type_registers,port,statut
device,,,,,,,
dummy1,modbus,localhost,big,big,holding,3502.0,actif
dummy2,modbus,localhost,big,little,holding,6500.0,actif
dummy_opcua,opcua,localhost,NaN,NaN,NaN,NaN,inactif


We load the modbus tables.

In [3]:
dummy_modbus_map1 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy1', index_col = 0)
dummy_modbus_map2 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy2', index_col = 0)
dummy_modbus_map1

,description,scale,unit,type,intaddress,size(mots),slave_unit,frequency
tag,,,,,,,,
PT_L5_H2O.HM05,pressure water line 5,10,mbar,float32,10,2,1,2
TT_L5_H2O.HM05,Temperature of water exchanger line 5,100,C,int32,26,2,1,2
PT_L7_H2O.HM05,pressure water line 7,100,mbar,float32,32,2,1,5
PT_L6_N2.HM05,pressure n2 line 6,10,mbar,float32,34,2,2,5
FT_L6_N2.HM05,Debit N2,10,g/s,float32,36,2,2,10
FT_L12_H2.HM05,debit H2,100,g/s,float32,68,2,1,10
V_C_TIME.CN,temps de fonctionnement,100,h,int16,70,1,1,60


# 2. Start the project with this configuration of devices

In [4]:
def generate_dummy_conf():
    return {
        'df_devices': dummy_df_devices,
        'modbus_maps' : {
            'dummy1':dummy_modbus_map1,
            'dummy2':dummy_modbus_map2
        },
        'tag_categories':{'dummy':{'water line':['PT_L5_H2O.HM05','PT_L7_H2O.HM05']}}
    }

In [5]:
from scadaUtils import Conf_generator 
conf = Conf_generator.Conf_generator_RT(project_name='dummy_project_gaia', function_generator=generate_dummy_conf,force_creation=True)

configuration file /home/dorian/dummy_project_gaia_user/conf_dummy_project_gaia.pkl found!
configuration file loaded !
generating configuration files and store it in : /home/dorian/dummy_project_gaia_user/conf_dummy_project_gaia.pkl
configuration file generated in  : 0.004526615142822266 seconds.
/home/dorian/gitLocals/scadaUtils/scadaUtils/Conf_generator.py,14 following credentials : {'host': 'localhost', 'port': '5432', 'user': 'test', 'password': 'test'}  to connect to the database are not valid.
Impossible to create the database realtime_data

/home/dorian/gitLocals/scadaUtils/scadaUtils/Conf_generator.py,287 ----------------------------------------------------------------------------------------------------  
Impossible to create the table : realtime_data 
Please make sure the database credentials are correct. 
Instanciation of your GAIA instance was killed. 
Solve this issue before instanciating again.
  ----------------------------------------------------------------------------

SystemExit: 

/home/dorian/inodiag/.dev/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
conf.dfplc

NameError: name 'conf' is not defined

In [ ]:
from scadaUtils import GAIA
# GAIA.GAIA?
gaia = GAIA.GAIA(project_name='dummy_project_gaia', function_generator=generate_dummy_conf,realtime=True,verbose=True,force_creation=True)

# 2. DUMP DATA 

## 2.1 Start the SIMULATOR of the modbus device (optionnal)
As we don't have the dummy device delivering data with the modbus protocole, we will create a simulator of the device based on the same modbus map and we will run the modbus server to serve data (random data). 
For your real project you can of course skip this step if the device is available on your network.

In [ ]:
from scadaUtils.Simulators import SimulatorModeBus

dummy_simulator1 = SimulatorModeBus(
   port = dummy_gaia.devices['dummy1'].port,
    modbus_map = dummy_gaia.devices['dummy1'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = dummy_gaia.devices['dummy1'].byte_order,
    wo = dummy_gaia.devices['dummy1'].word_order)

dummy_simulator2 = SimulatorModeBus(
   port = dummy_gaia.devices['dummy2'].port,
    modbus_map = dummy_gaia.devices['dummy2'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = dummy_gaia.devices['dummy2'].byte_order,
    wo = dummy_gaia.devices['dummy2'].word_order)

dummy_simulator.start()

In [ ]:
# Start dumping data
# param bool park_on_time: if True, it will wait for the parking to start at a round time.
dummy_gaia.start_dumping(park_on_time=False)

In [ ]:
dummy_gaia.read_db()

In [ ]:
dummy_gaia.stop_dumping()

In [ ]:
dummy_gaia.run_GUI()

- to deploy it to the world, open the port of the app on your routeur and use host='0.0.0.0'

As Flask says it is not a reliable as a production environment. It is better to use wsgi and gunicorn to run the app.
- more convenient is to control the process with with **systemctl** on a Linux server. 
- Then use **nginx**(or apache) to make the redirection via a reverse proxy configured in a <myapp>.conf file in the nginx folders *sites_enabled* and *sites-available* to be able to map your domain name or subdomain name to the app. A guide is to be found [here](https://github.com/dorianSylfen/smallpower).